In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# 상태 변수
user_answers = []
correct_answers = []
labeled_images = []
game_running = False

# CIFAR-10 데이터 로드
(train_x, _), (_, _) = tf.keras.datasets.cifar10.load_data()
train_x = (train_x.astype('float32') - 127.5) / 127.5

# GAN Generator 불러오기
generator = tf.keras.models.load_model('generator.h5')

# 이미지 준비 함수
def prepare_game():
    global labeled_images, user_answers, correct_answers, game_running
    user_answers = []
    correct_answers = []
    game_running = True

    noise = tf.random.normal([10, 100])
    fake_images = generator(noise, training=False)
    real_images = train_x[np.random.choice(len(train_x), 10, replace=False)]
    labeled_images = [(img, 'real') for img in real_images] + [(img, 'fake') for img in fake_images]
    random.shuffle(labeled_images)

# 게임 시작 화면
def start_screen():
    global game_running
    game_running = False
    clear_output(wait=True)

    st_title = widgets.HTML("<h2> 진진GAN 이미지 판별 게임</h2><p>real vs fake 이미지 맞히기!</p>")
    btn_start = widgets.Button(description="게임 시작", button_style='success')

    def on_start(b):
        prepare_game()
        show_image(0)

    btn_start.on_click(on_start)
    display(st_title, btn_start)

# 이미지 보여주기
def show_image(index):
    clear_output(wait=True)
    img, label = labeled_images[index]
    correct_answers.append(label)

    plt.imshow((img + 1) / 2)
    plt.title(f"이미지 {index + 1}")
    plt.axis('off')
    plt.show()

    def on_choice(btn):
        user_answers.append(btn.description)
        if index + 1 < 7:
            show_image(index + 1)
        else:
            show_result()

    def on_cancel(b):
        clear_output(wait=True)
        start_screen()

    btn_real = widgets.Button(description="real", button_style='success')
    btn_fake = widgets.Button(description="fake", button_style='danger')
    btn_real.on_click(on_choice)
    btn_fake.on_click(on_choice)

    btn_cancel = widgets.Button(description="게임 취소", button_style='warning')
    btn_cancel.on_click(on_cancel)

    display(widgets.HBox([btn_real, btn_fake]), btn_cancel)

# 결과 보기 + 다시 시작 버튼
def show_result():
    global game_running
    game_running = False
    clear_output(wait=True)

    score = sum([u == c for u, c in zip(user_answers, correct_answers)])
    for i, (u, c) in enumerate(zip(user_answers, correct_answers)):
        result = "✅" if u == c else f"❌ (정답: {c})"
        print(f"이미지 {i+1}: 선택={u} → {result}")
    print(f"\n🎯 최종 점수: {score} / 7   정확도: {score / 7 * 100:.1f}%")

    btn_restart = widgets.Button(description="게임 시작", button_style='primary')
    btn_restart.on_click(lambda b: start_screen())
    display(btn_restart)

# 시작
start_screen()


HTML(value='<h2> 진진GAN 이미지 판별 게임</h2><p>real vs fake 이미지 맞히기!</p>')

Button(button_style='success', description='게임 시작', style=ButtonStyle())